In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, f1_score

import keras
import tensorflow as tf

from nltk.corpus import stopwords
import nltk
import re

from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("data/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

print("Train shape: {}".format(train.shape))
print("Test shape: {}".format(test.shape))
print("Unlabeled train shape: {}".format(unlabeled_train.shape))

Train shape: (25000, 3)
Test shape: (25000, 2)
Unlabeled train shape: (50000, 2)


In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [5]:
# Download the punkt tokenizer for sentence splitting  
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [6]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/var/folders/jw/pk9yd5v978d_6j_wky4ptw480000gn/T/ipykernel_24102/1919973641.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
/var/folders/jw/pk9yd5v978d_6j_wky4ptw480000gn/T/ipykernel_24102/1919973641.py:11: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


Parsing sentences from unlabeled set


In [7]:
print(len(sentences))

796172


In [11]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences,
                          workers=num_workers,
                          min_count = min_word_count,
                          window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2023-11-05 17:21:48,687 : INFO : collecting all words and their counts
2023-11-05 17:21:48,688 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-05 17:21:48,760 : INFO : PROGRESS: at sentence #10000, processed 225664 words, keeping 17775 word types
2023-11-05 17:21:48,832 : INFO : PROGRESS: at sentence #20000, processed 451738 words, keeping 24945 word types
2023-11-05 17:21:48,875 : INFO : PROGRESS: at sentence #30000, processed 670859 words, keeping 30027 word types


Training model...


2023-11-05 17:21:48,926 : INFO : PROGRESS: at sentence #40000, processed 896841 words, keeping 34335 word types
2023-11-05 17:21:48,975 : INFO : PROGRESS: at sentence #50000, processed 1116082 words, keeping 37751 word types
2023-11-05 17:21:49,017 : INFO : PROGRESS: at sentence #60000, processed 1337544 words, keeping 40711 word types
2023-11-05 17:21:49,061 : INFO : PROGRESS: at sentence #70000, processed 1560307 words, keeping 43311 word types
2023-11-05 17:21:49,110 : INFO : PROGRESS: at sentence #80000, processed 1779516 words, keeping 45707 word types
2023-11-05 17:21:49,155 : INFO : PROGRESS: at sentence #90000, processed 2003714 words, keeping 48121 word types
2023-11-05 17:21:49,197 : INFO : PROGRESS: at sentence #100000, processed 2225465 words, keeping 50190 word types
2023-11-05 17:21:49,240 : INFO : PROGRESS: at sentence #110000, processed 2444323 words, keeping 52058 word types
2023-11-05 17:21:49,285 : INFO : PROGRESS: at sentence #120000, processed 2666488 words, keepin